In [17]:
%pip install langchain langchain-openai langchainhub --quiet

Note: you may need to restart the kernel to use updated packages.


# example from https://python.langchain.com/v0.1/docs/modules/agents/agent_types/react/

In [18]:
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent
from langchain_openai import OpenAI

In [19]:
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/react")

from pprint import pprint
pprint(prompt.template)
#a=prompt.format_prompt(context="some context",question="some question")
#for msg in a.messages:
#    print(msg.content)

('Answer the following questions as best you can. You have access to the '
 'following tools:\n'
 '\n'
 '{tools}\n'
 '\n'
 'Use the following format:\n'
 '\n'
 'Question: the input question you must answer\n'
 'Thought: you should always think about what to do\n'
 'Action: the action to take, should be one of [{tool_names}]\n'
 'Action Input: the input to the action\n'
 'Observation: the result of the action\n'
 '... (this Thought/Action/Action Input/Observation can repeat N times)\n'
 'Thought: I now know the final answer\n'
 'Final Answer: the final answer to the original input question\n'
 '\n'
 'Begin!\n'
 '\n'
 'Question: {input}\n'
 'Thought:{agent_scratchpad}')


In [20]:


# Choose the LLM to use
#llm = OpenAI()
from _callbacks import PrettyPrintCallbackHandler
callback = PrettyPrintCallbackHandler()

import _models, _settings
llm=_models.get_llm(_settings.MODEL_NAME, callbacks=[callback])



In [21]:
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool


tools = []
class BashValidatorInput(BaseModel):
    code: str = Field(description="The code to validate")

import _tools


bash_validator_tool = StructuredTool.from_function(
    func=_tools.bash_validate,
    name="bash_validator",
    description="Check if the code is valid bash code",
    args_schema=BashValidatorInput,
  #  return_direct=True,
    handle_tool_eror=True
)


class MarkdownValidatorInput(BaseModel):
    code: str = Field(description="The code to validate")



markdown_validator_tool = StructuredTool.from_function(
    func=_tools.markdown_validate,
    name="markdown_validator",
    description="Check if the code is valid markdown",
    args_schema=MarkdownValidatorInput,
  #  return_direct=True,
    handle_tool_eror=True
)


tools.append(bash_validator_tool)
tools.append(markdown_validator_tool)

In [22]:

# Construct the ReAct agent
agent = create_react_agent(llm, tools, prompt)

In [23]:
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools,
         verbose=True,return_intermediate_steps=True,
           callbacks=[callback], max_iterations=15)

In [24]:
agent_executor.invoke({"input": "Write me a shell script to list the files in the current directory. Output the script as markdown bash code block"})



[chain][start] - inputs {'input': 'Write me a shell script to list the files in the current directory. Output the script as markdown bash code block'}


> Entering new AgentExecutor chain...
[agent][action] - action: tool='None' tool_input='None' log='To answer this question, I need to write a bash script that lists the files in the current directory. Then, I need to format it as a markdown bash code block.\nAction: None\nAction Input: None'
To answer this question, I need to write a bash script that lists the files in the current directory. Then, I need to format it as a markdown bash code block.
Action: None
Action Input: NoneNone is not a valid tool, try one of [bash_validator, markdown_validator].[agent][action] - action: tool='bash_validator' tool_input='ls -l' log='I need to write the bash script first and then validate it using the bash_validator tool.\nAction: bash_validator\nAction Input: "ls -l"'
I need to write the bash script first and then validate it using the bash_valida

{'input': 'Write me a shell script to list the files in the current directory. Output the script as markdown bash code block',
 'output': 'The shell script to list the files in the current directory, outputted as a markdown bash code block is:\n\n```bash\nls -l\n```',
 'intermediate_steps': [(AgentAction(tool='None', tool_input='None', log='To answer this question, I need to write a bash script that lists the files in the current directory. Then, I need to format it as a markdown bash code block.\nAction: None\nAction Input: None'),
   'None is not a valid tool, try one of [bash_validator, markdown_validator].'),
  (AgentAction(tool='bash_validator', tool_input='ls -l', log='I need to write the bash script first and then validate it using the bash_validator tool.\nAction: bash_validator\nAction Input: "ls -l"'),
   'The code is valid bash code'),
  (AgentAction(tool='markdown_validator', tool_input='```bash\\nls -l\\n```', log='The bash script is valid. Now I need to format it as a mar